In [ ]:
from keras.preprocessing import sequence
from keras.preprocessing import text
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding, LSTM
from keras.layers import Conv1D, Flatten
from keras.preprocessing import text
from keras.models import Sequential,Model
from keras.layers import Dense ,Activation,MaxPool1D,Conv1D,Flatten,Dropout,Activation,Dropout,Input,Lambda,concatenate
from keras.utils import np_utils
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
import nltk
import csv
import pandas as pd

from keras.preprocessing import text as keras_text, sequence as keras_seq

In [ ]:
data = pd.read_csv('/content/drive/My Drive/ML Internship IIITD/train.csv')
pd.set_option('display.max_colwidth',80)
data.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in share market in india?,What is the step by step guide to invest in share market?,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Diamond?,What would happen if the Indian government stole the Kohinoor (Koh-i-Noor) d...,0
2,2,5,6,How can I increase the speed of my internet connection while using a VPN?,How can Internet speed be increased by hacking through DNS?,0
3,3,7,8,Why am I mentally very lonely? How can I solve it?,"Find the remainder when [math]23^{24}[/math] is divided by 24,23?",0
4,4,9,10,"Which one dissolve in water quikly sugar, salt, methane and carbon di oxide?",Which fish would survive in salt water?,0


In [ ]:
data.shape

(404290, 6)

In [ ]:
print(data.dtypes)
print(data['question1'].dtypes)
print(data['question2'].dtypes)
type(data['question1'])

id               int64
qid1             int64
qid2             int64
question1       object
question2       object
is_duplicate     int64
dtype: object
object
object


pandas.core.series.Series

# Setting target or labelfor each input

In [ ]:
label_oneDimension=data['is_duplicate']

In [ ]:
label_oneDimension.head(2)

0    0
1    0
Name: is_duplicate, dtype: int64

In [ ]:
import numpy as np
from keras.utils.np_utils import to_categorical
label_twoDimension = to_categorical(data['is_duplicate'], num_classes=2)

In [ ]:
label_twoDimension[0:1]

array([[1., 0.]], dtype=float32)

In [ ]:
question_one=data['question1'].astype(str)
print(question_one.head())

0              What is the step by step guide to invest in share market in india?
1                             What is the story of Kohinoor (Koh-i-Noor) Diamond?
2       How can I increase the speed of my internet connection while using a VPN?
3                              Why am I mentally very lonely? How can I solve it?
4    Which one dissolve in water quikly sugar, salt, methane and carbon di oxide?
Name: question1, dtype: object


In [ ]:
question_two=data['question2'].astype(str)
print(question_two.head())

0                          What is the step by step guide to invest in share market?
1    What would happen if the Indian government stole the Kohinoor (Koh-i-Noor) d...
2                        How can Internet speed be increased by hacking through DNS?
3                  Find the remainder when [math]23^{24}[/math] is divided by 24,23?
4                                            Which fish would survive in salt water?
Name: question2, dtype: object


# Reading test data and preprocessing

In [ ]:
#Data reading
'''
data_test = pd.read_csv('drive/My Drive/Summer Internship 2020 July/My Test File/Sunil/test.csv')
data_test_sample=data_test.dropna()
#data_test_sample=data_test_sample.head(100)
data_test_sample.head()
'''

"\ndata_test = pd.read_csv('drive/My Drive/Summer Internship 2020 July/My Test File/Sunil/test.csv')\ndata_test_sample=data_test.dropna()\n#data_test_sample=data_test_sample.head(100)\ndata_test_sample.head()\n"

In [ ]:
'''
question_one_test=data_test_sample['question1'].astype(str)
print(question_one_test.head())
'''

"\nquestion_one_test=data_test_sample['question1'].astype(str)\nprint(question_one_test.head())\n"

In [ ]:
'''
question_two_test=data_test_sample['question2'].astype(str)
print(question_two_test.head())
'''

"\nquestion_two_test=data_test_sample['question2'].astype(str)\nprint(question_two_test.head())\n"

# Fitting text on a single tokenized object

In [ ]:
from keras.preprocessing.text import Tokenizer
tok_all = Tokenizer(filters='!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~', lower=True, char_level = False)
tok_all.fit_on_texts(question_one+question_two)
#tok_all.fit_on_texts(question_one+question_two+question_one_test+question_two_test)


In [ ]:
l=tok_all.word_index
print(l)

{'the': 1, 'what': 2, 'is': 3, 'i': 4, 'how': 5, 'a': 6, 'to': 7, 'in': 8, 'do': 9, 'of': 10, 'are': 11, 'and': 12, 'can': 13, 'for': 14, 'you': 15, 'why': 16, 'my': 17, 'it': 18, 'best': 19, 'on': 20, 'does': 21, 'or': 22, 'which': 23, 'be': 24, 'if': 25, 's': 26, 'some': 27, 'have': 28, 'that': 29, 'with': 30, 'get': 31, 'should': 32, 'an': 33, 'from': 34, 'your': 35, 'india': 36, 'will': 37, 'people': 38, 'when': 39, 'who': 40, 'like': 41, 'at': 42, 'good': 43, 'there': 44, 'would': 45, 'as': 46, 't': 47, 'about': 48, 'not': 49, 'between': 50, 'one': 51, 'most': 52, 'we': 53, 'quora': 54, 'make': 55, 'way': 56, 'did': 57, 'where': 58, 'by': 59, 'any': 60, 'was': 61, 'me': 62, 'life': 63, 'so': 64, 'they': 65, 'after': 66, 'time': 67, 'this': 68, 'money': 69, 'know': 70, 'difference': 71, 'has': 72, 'learn': 73, 'am': 74, 'new': 75, 'use': 76, 'much': 77, 'someone': 78, 'their': 79, 'think': 80, 'many': 81, 'work': 82, 'all': 83, 'indian': 84, 'trump': 85, 'than': 86, 'more': 87, 'ev

In [ ]:
vocabulary_all=len(tok_all.word_counts)
print(vocabulary_all)

89983


# Train data Sequencing and Encoding

In [ ]:
#Encoding question 1
encoded_q1=tok_all.texts_to_sequences(question_one)
print(question_one[0])
encoded_q1[0]

What is the step by step guide to invest in share market in india?


[2, 3, 1, 1222, 59, 1222, 2566, 7, 579, 8, 763, 384, 8, 36]

In [ ]:
#Encoding question 2
encoded_q2=tok_all.texts_to_sequences(question_two)
print(question_two[0])
encoded_q2[0]

What is the step by step guide to invest in share market?


[2, 3, 1, 1222, 59, 1222, 2566, 7, 579, 8, 763, 384]

# Pre-Padding on Train data

In [ ]:
#####Padding encoded sequence of words
from keras.preprocessing import sequence
max_length=50
padded_docs_q1 = sequence.pad_sequences(encoded_q1, maxlen=max_length, padding='pre')

In [ ]:
#####Padding encoded sequence of words
from keras.preprocessing import sequence
max_length=50
padded_docs_q2 = sequence.pad_sequences(encoded_q2, maxlen=max_length, padding='pre')

In [ ]:
print(question_two[0])
print(padded_docs_q2[0])

What is the step by step guide to invest in share market?
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    2    3    1 1222   59 1222 2566    7  579    8
  763  384]


# Encoding on Test data

In [ ]:
'''
#Encoding question 1
encoded_q1_test=tok_all.texts_to_sequences(question_one_test)
print(question_one_test[0])
encoded_q1_test[0]
'''

'\n#Encoding question 1\nencoded_q1_test=tok_all.texts_to_sequences(question_one_test)\nprint(question_one_test[0])\nencoded_q1_test[0]\n'

In [ ]:
'''#Encoding question 1
encoded_q2_test=tok_all.texts_to_sequences(question_two_test)
print(question_two_test[0])
encoded_q2_test[0]'''

'#Encoding question 1\nencoded_q2_test=tok_all.texts_to_sequences(question_two_test)\nprint(question_two_test[0])\nencoded_q2_test[0]'

# Pre-Padding on test data

In [ ]:
'''#####Padding encoded sequence of words
padded_docs_q1_test = sequence.pad_sequences(encoded_q1_test, maxlen=max_length, padding='pre')
padded_docs_q2_test = sequence.pad_sequences(encoded_q2_test, maxlen=max_length, padding='pre')'''

"#####Padding encoded sequence of words\npadded_docs_q1_test = sequence.pad_sequences(encoded_q1_test, maxlen=max_length, padding='pre')\npadded_docs_q2_test = sequence.pad_sequences(encoded_q2_test, maxlen=max_length, padding='pre')"

# Reading Embedding Vector from Glove

In [ ]:
import os
import numpy as np

In [ ]:
embeddings_index = {}
f = open('drive/My Drive/ML Internship IIITD/Copy of glove.6B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

FileNotFoundError: ignored

In [ ]:
embedding_matrix = np.zeros((vocabulary_all+1, 300))
for word, i in tok_all.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

# Defining Input Shape for Model

In [ ]:
Question1_shape= Input(shape=[max_length])
Question1_shape.shape

In [ ]:
Question2_shape= Input(shape=[max_length])
Question2_shape.shape

# Embedding Layer

In [ ]:
Embedding_Layer = Embedding(vocabulary_all+1,300,weights=[embedding_matrix], input_length=max_length, trainable=False)

# CNN Network

In [ ]:
CNN1_network=Sequential([Embedding_Layer,
                        Conv1D(32,3,activation="relu",padding='same'),
                       Dropout(0.2),
                       MaxPool1D(2),
                       Flatten(),
                       Dense(128,activation="linear"),
                       Dropout(0.3)
                       
                       ])

# Printing Model summary

In [ ]:
CNN1_network.summary()

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(CNN1_network, to_file='CNN1_network.png', show_shapes=True, show_layer_names=True)

# create siamese network from CNN model and store output feature vectors

In [ ]:
Question1_CNN_feature=CNN1_network(Question1_shape)
Question2_CNN_feature=CNN1_network(Question2_shape)

# Adding and multiplying features obtained from Siamese CNN network

In [ ]:
from keras import backend as K
from keras.optimizers import Adam

In [ ]:
lamda_function=Lambda(lambda tensor:K.abs(tensor[0]-tensor[1]),name="Absolute_distance")
abs_distance_vector=lamda_function([Question1_CNN_feature,Question2_CNN_feature])

lamda_function2=Lambda(lambda tensor:K.abs(tensor[0]*tensor[1]),name="Hamadard_multiplication")
hamadard_vector=lamda_function2([Question1_CNN_feature,Question2_CNN_feature])

# Adding abs_distance_vector and hamadard_vector

In [ ]:
from keras.layers import Add

In [ ]:
added_vecotr = Add()([abs_distance_vector, hamadard_vector])

# Final Model prediction

In [ ]:
predict=Dense(2,activation="sigmoid")(added_vecotr)

# Creating sequential model using Model() class and compilation

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score
Siamese1_Network=Model(inputs=[Question1_shape,Question2_shape],outputs=predict)
Siamese1_Network.compile(loss = "binary_crossentropy", optimizer=Adam(lr=0.00003), metrics=["accuracy"])

In [ ]:
Siamese1_Network.summary()

# Plot model

In [ ]:
from keras.utils import plot_model
plot_model(Siamese1_Network, to_file='Siamese1_Network.png',show_shapes=True, show_layer_names=True)

# Setting hyperparameter for training

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau,ModelCheckpoint
earlystopper = EarlyStopping(patience=8, verbose=1)
#checkpointer = ModelCheckpoint(filepath = 'cnn_model_one_.{epoch:02d}-{val_loss:.6f}.hdf5',
 #                              verbose=1,
 #                              save_best_only=True, save_weights_only = True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.9,
                              patience=2, min_lr=0.00001, verbose=1)

# Data split into train and validation set

In [ ]:
# Splitting data into train and test
from sklearn.model_selection import train_test_split
q1_train, q1_val,q2_train, q2_val, label_train, label_val, label_oneD_train, label_oneD_val = train_test_split(padded_docs_q1,padded_docs_q2, label_twoDimension, label_oneDimension, test_size=0.30,
random_state=42)

# Model fitting or training

In [ ]:
history = Siamese1_Network.fit([q1_train,q2_train],label_train,
                    batch_size=32,epochs=100,validation_data=([q1_val,q2_val],label_val),callbacks=[earlystopper, reduce_lr])

# Model Prediction

In [ ]:
Siamese1_Network_predictions = Siamese1_Network.predict([q1_val,q2_val])
#Siamese1_Network_predictions = Siamese1_Network.predict([padded_docs_q1_test,padded_docs_q2_test])

In [ ]:
#Siamese1_Network_predictions_testData = Siamese1_Network.predict([padded_docs_q1_test,padded_docs_q1_test])

# Log loss

In [ ]:
from sklearn.metrics import log_loss
log_loss_val= log_loss(label_val,Siamese1_Network_predictions)
log_loss_val

# Classification report

In [ ]:
predictions = np.zeros_like(Siamese1_Network_predictions)
predictions[np.arange(len(Siamese1_Network_predictions)), Siamese1_Network_predictions.argmax(1)] = 1

In [ ]:
predictionInteger=(np.argmax(predictions, axis=1))
#print('np.argmax(a, axis=1): {0}'.format(np.argmax(predictions, axis=1)))
predictionInteger

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(label_val,predictions))

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
print ("Precision, Recall, F1_score : macro ",precision_recall_fscore_support(label_oneD_val,predictionInteger, average='macro'))
print ("Precision, Recall, F1_score : micro ",precision_recall_fscore_support(label_oneD_val,predictionInteger, average='micro'))
print ("Precision, Recall, F1_score : weighted ",precision_recall_fscore_support(label_oneD_val,predictionInteger, average='weighted'))

# Final train and val loss

In [ ]:
min_val_loss = min(history.history["val_loss"])
min_train_loss = min(history.history["loss"])
max_val_acc = max(history.history["val_accuracy"])
max_train_acc = max(history.history["accuracy"])
print("min_train_loss=%g, min_val_loss=%g, max_train_acc=%g, max_val_acc=%g" % (min_train_loss,min_val_loss,max_train_acc,max_val_acc))

# Plot epoch Vs loss

In [ ]:
from matplotlib import pyplot as plt
plt.plot(history.history["loss"],color = 'red', label = 'train_loss')
plt.plot(history.history["val_loss"],color = 'blue', label = 'val_loss')
plt.title('Loss Visualisation')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig('1Layer_CNN_lossPlot_siamese.pdf',dpi=1000)

from google.colab import files
files.download('1Layer_CNN_lossPlot_siamese.pdf') 

# Plot Epoch Vs Accuracy

In [ ]:
plt.plot(history.history["accuracy"],color = 'red', label = 'train_accuracy')
plt.plot(history.history["val_accuracy"],color = 'blue', label = 'val_accuracy')
plt.title('Accuracy Visualisation')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig('1Layer_CNN_accuracyPlot_siamese.pdf',dpi=1000)
files.download('1Layer_CNN_accuracyPlot_siamese.pdf') 

# Area Under Curve- ROC

In [ ]:
#pred_test = Siamese1_Network.predict([padded_docs_q1_test,padded_docs_q2_test])
pred_train = Siamese1_Network.predict([q1_train,q2_train])
pred_val = Siamese1_Network.predict([q1_val,q2_val])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp

def plot_AUC_ROC(y_true, y_pred):
    n_classes = 2 #change this value according to class value

    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_true[:, i], y_pred[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(y_true.ravel(), y_pred.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
############################################################################################
    lw = 2
    # Compute macro-average ROC curve and ROC area

    # First aggregate all false positive rates
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

    # Then interpolate all ROC curves at this points
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(n_classes):
        mean_tpr += interp(all_fpr, fpr[i], tpr[i])

    # Finally average it and compute AUC
    mean_tpr /= n_classes

    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

    # Plot all ROC curves
    plt.figure()
    
    plt.plot(fpr["micro"], tpr["micro"],
            label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
            color='deeppink', linestyle=':', linewidth=4)

    plt.plot(fpr["macro"], tpr["macro"],
            label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
            color='navy', linestyle=':', linewidth=4)
    
    colors = cycle(['aqua', 'darkorange'])
    #classes_list1 = ["DE","NE","DK"]
    classes_list1 = ["Non-duplicate","Duplicate"]
    for i, color,c in zip(range(n_classes), colors,classes_list1):
        plt.plot(fpr[i], tpr[i], color=color, lw=lw,
                label='{0} (AUC = {1:0.2f})'
                ''.format(c, roc_auc[i]))

    plt.plot([0, 1], [0, 1], 'k--', lw=lw)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic curve')
    plt.legend(loc="lower right")
    #plt.show()
    plt.savefig('1Layer_CNN_RocPlot_siamese.pdf',dpi=1000)
    files.download('1Layer_CNN_RocPlot_siamese.pdf')

    # Plot of a ROC curve for a specific class
    '''
    plt.figure()
    lw = 2
    plt.plot(fpr[0], tpr[0], color='darkorange',
            lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[0])
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()
    '''

In [ ]:
plot_AUC_ROC(label_val,pred_val)

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score

In [ ]:
auc_val = roc_auc_score(label_val,pred_val)
accuracy_val = accuracy_score(label_val,pred_val>0.5)
auc_train = roc_auc_score(label_train,pred_train)
accuracy_train = accuracy_score(label_train,pred_train>0.5)

print("auc_train=%g, auc_val=%g, accuracy_train=%g, accuracy_val=%g" % (auc_train, auc_val, accuracy_train, accuracy_val))

In [ ]:
'''
fpr_train, tpr_train, thresholds_train = roc_curve(label_train,pred_train)
fpr_test, tpr_test, thresholds_test = roc_curve(label_val,pred_val)

#fpr_train, tpr_train, thresholds_train = roc_curve(label_oneD_train,pred_train_final)
#fpr_test, tpr_test, thresholds_test = roc_curve(label_oneD_val,pred_val_final)

plt.plot(fpr_train,tpr_train, color="blue", label="train roc, auc=%g" % (auc_train,))
plt.plot(fpr_test,tpr_test, color="green", label="val roc, auc=%g" % (auc_val,))

plt.plot([0,1], [0,1], color='orange', linestyle='--')

plt.xticks(np.arange(0.0, 1.1, step=0.1))
plt.xlabel("Flase Positive Rate", fontsize=15)

plt.yticks(np.arange(0.0, 1.1, step=0.1))
plt.ylabel("True Positive Rate", fontsize=15)

plt.title('ROC Curve Analysis', fontweight='bold', fontsize=15)
plt.legend(prop={'size':13}, loc='lower right')
plt.savefig('AUC_CURVE_cnn4.pdf',dpi=1000)
#files.download('AUC_CURVE_cnn4.pdf')
'''